# 0. Default start

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [3]:
np.random.seed(42)

In [4]:
df = pd.read_csv('train_dataset_Самолет.csv')

C:\Users\Artem\AppData\Local\Temp\ipykernel_14232\1363469232.py:1: DtypeWarning: Columns (51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,131,132,133,135,136,139,140,141,142,143,144,145,146,147,148,149,151,152,155,156,157,158,159,160,161,163,164,165,167,168,169,171,172,173,174,175,176,177,178,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,223,224,227,239,240,241,242,243,244,245,246,459,460,461,467,468,469,515,516,523,524,531,532,539,540,547,548,555,556,563,564,565,567,571,572,573,575,579,580,581,582,583,587,588,595,596,603,604,611,612,613,615,795,796,797,799,821,822,825,826,1031,1032,1033,1034,1035,1036,1037,1039,1040,1041,1042,1043,1044,1045,1047,1048,1049,1050,1051,1052,1055,1056,1057,1058,1059,1060,1063,1064,1065,1067,1068,1069,1072,1073,1074,1075,1076,1077,1078,1172,1173,1174,1176,1177,1178,1179,1180,1181,1182,1275,1276,1277,1278,1279,1280,1281,1282,14

In [5]:
def device(data):
    columns = [f'col{n}' for n in range(529, 544)]
    new_feature_name = 'device'
    most_common_value = data[columns].mode(axis=1).iloc[:, 0]
    data[new_feature_name] = most_common_value
    return data

In [6]:
def info_redirect(data):
    columns = [f'col{n}' for n in range(561, 576)]
    mail_info = 'info_mail'
    most_common_value = any(['sms' in data[columns], 'email' in data[columns]])
    data[mail_info] = int(most_common_value)
    
    invite_count = 'sale_invite_count'
    most_common_value = (data[columns].notnull()).sum(axis=1)
    data[invite_count] = most_common_value
    return data

In [7]:
def generate_new_features(data):
    """
    Генерирует новые признаки на основе заданных колонок.
    :param data: pandas.DataFrame, содержащий данные.
    :param columns: список, содержащий названия колонок для генерации новых признаков.
    :return: pandas.DataFrame с добавленными новыми признаками.
    """
    columns = [f'col{n}' for n in range(465, 473)]
    new_feature_name = 'filter' + '_most_common'
    most_common_value = data[columns].mode(axis=1).iloc[:, 0]
    data[new_feature_name] = most_common_value

    count_feature_name = 'filter' + '_count'
    count_of_most_common = (data[columns] == most_common_value[..., None]).sum(axis=1)
    data[count_feature_name] = count_of_most_common

    non_zero_feature_name = 'filter' + '_non_zero_count'
    non_zero_count = (data[columns] != 0).sum(axis=1)
    data[non_zero_feature_name] = non_zero_count

    return data

In [8]:
def calculate_quantitative_metrics(data):
    """
    Вычисляет количественные метрики на основе заданных колонок для каждой строки в датафрейме.
    :param data: pandas.DataFrame, содержащий данные.
    :param columns: список, содержащий названия колонок для вычисления метрик.
    :return: pandas.DataFrame с добавленными количественными метриками.
    """
    columns = [f'col{n}' for n in range(457,465)]
    # Количество уникальных страниц
    unique_value = data[columns].nunique(axis=1)
    data['unique_pages_count'] = unique_value

    # Количество общих страниц
    count_feature_name = 'visit_site' + '_count'
    count_of_most_common = (data[columns].notnull()).sum(axis=1)
    data[count_feature_name] = count_of_most_common

    return data

In [9]:
df = device(df)
df = info_redirect(df)
df = generate_new_features(df)
df = calculate_quantitative_metrics(df)

C:\Users\Artem\AppData\Local\Temp\ipykernel_14232\3852941217.py:14: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  count_of_most_common = (data[columns] == most_common_value[..., None]).sum(axis=1)


# 1. Data analyze

In [10]:
def count_missing_values(df):
    """
    Функция для подсчета пропусков в указанном столбце DataFrame. 
    Параметры:
    df (pd.DataFrame): Исходный DataFrame. 
    Возвращает:
    float: Процент пропусков в столбце."""
    total_rows = df.shape[0]  # Общее количество строк в DataFrame
    missing_values = df.isnull().sum()  # Подсчет пропусков в столбце

    # Вычисление процента пропусков
    missing_percentage = (missing_values / total_rows) * 100

    return missing_percentage

In [11]:
df = df.replace({'nan': None})

In [12]:
df.shape

(14456, 2674)

In [13]:
df = df.T.drop_duplicates().T
df.shape

(14456, 2221)

In [14]:
miss_pos_col = count_missing_values(df[df['target'] == 1])
threshold = 75 # percent value in col for drop
to_drop = miss_pos_col[miss_pos_col >= threshold].index.tolist()
len(to_drop)

1878

In [15]:
df = df.drop(to_drop, axis=1)

In [16]:
df.shape

(14456, 343)

In [17]:
def intersection_col(df, list_cols):
    if isinstance(list_cols, list):
        return list(set(df.columns.tolist()) & set(list_cols))
    else:
        return list(set(df.columns.tolist()) & set(list_cols.columns.tolist()))

In [18]:
cat_all_columns = ['col520', 'col528', 'col536',
 'col544', 'col552', 'col592', 'col600', 'col608', 'col1454',
                  'device', 'info_mail', 'filter_most_common']
to_drop_cat = ['col552', 'col1454']

In [19]:
cat_cols = intersection_col(df, cat_all_columns)
to_drop = list(set(to_drop_cat) & set(cat_cols))
cat_cols = list(set(cat_cols) - set(to_drop))
len(cat_cols)

1

In [20]:
df = df.drop(to_drop, axis=1)

In [21]:
df.shape

(14456, 342)

In [22]:
cols = df.columns.tolist()
cols.remove('client_id'), cols.remove('target'), cols.remove('report_date')
df[cols] = df[cols].astype(np.float64)

category_cols = []

for col in df.columns.tolist():
    if col == 'report_date':
        continue
    if df[col].max() <= 40:
        category_cols.append(col)
category_cols.remove('col2663'), category_cols.remove('target')

(None, None)

In [23]:
# df[category_cols] = df[category_cols].astype(str)

# 2. Model builder

In [24]:
from sklearn.metrics import classification_report
from eli5.sklearn import PermutationImportance
from lightgbm import LGBMClassifier
from category_encoders import CatBoostEncoder

In [25]:
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
    average_precision_score,
    log_loss,
    )
def evaluate_metrics(y_true, y_pred):
    roc_auc = roc_auc_score(y_true, y_pred) 
    accuracy = accuracy_score(y_true, (y_pred > 0.5).astype(int)) 
    precision = precision_score(y_true, (y_pred > 0.5).astype(int)) 
    recall = recall_score(y_true, (y_pred > 0.5).astype(int))
    f1 = f1_score(y_true, (y_pred > 0.5).astype(int)) 
    # confusion = confusion_matrix(y_true, (y_pred > 0.5).astype(int))
    pr_auc = average_precision_score(y_true, y_pred) 
    logloss = log_loss(y_true, y_pred)

    print(f"ROC AUC: {roc_auc}", end='; ')
    print(f"Accuracy: {accuracy}", end='; ')
    print(f"Precision: {precision}", end='; ')
    print(f"Recall: {recall}", end='; ')
    print(f"F1 Score: {f1}", end='; ')
    print(f"PR AUC: {pr_auc}", end='; ')
    print(f"Log Loss: {logloss}")

In [26]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, ClassifierMixin

from eli5.sklearn import PermutationImportance
from imblearn.over_sampling import SMOTE
import optuna
from optuna.integration import LightGBMPruningCallback
import lightgbm as lgb
from copy import deepcopy

    
class FeatureSelector(BaseEstimator, ClassifierMixin):
    def __init__(self, features):
        """
        Класс для отбора заданных признаков.
        
        Параметры:
        features (list): Список заданных признаков.
        """
        self.features = features
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        """
        Метод для отбора заданных признаков из датасета.
        
        Параметры:
        df (pd.DataFrame): Входной датасет.
        
        Возвращает:
        pd.DataFrame: Датасет, содержащий только отобранные признаки.
        
        Исключения:
        ValueError: Генерируется, если какие-то из заданных признаков отсутствуют во входном датасете.
        """
        # Проверяем наличие всех заданных признаков во входном датасете
        missing_features = set(self.features) - set(df.columns)
        if missing_features:
            raise ValueError(f"Признаки {missing_features} отсутствуют во входном датасете.")
        
        # Отбираем заданные признаки из датасета
        selected_df = X[self.features]
        
        return selected_df

def objective(trial, trf, df_train, df_val, target_col='target'):
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
        'min_sum_hessian_in_leaf ': trial.suggest_float('min_sum_hessian_in_leaf', 0.0, 10.0),
        'is_unbalance': trial.suggest_categorical('is_unbalance', [True]),
        'random_state': trial.suggest_categorical('random_state', [42])
    }
    pruning_callback = LightGBMPruningCallback(trial, "auc")
    trf = trf.fit(df_train, df_train[target_col])

    model = lgb.train(params, lgb.Dataset(trf.transform(df_train), label=df_train[target_col]), num_boost_round=1000,
                    valid_sets=[lgb.Dataset(trf.transform(df_val), label=df_val[target_col])], 
                    early_stopping_rounds=100, verbose_eval=False, callbacks=[pruning_callback])
   
    y_pred = model.predict(trf.transform(df_val))

    evaluate_metrics(df_val[target_col], y_pred)
    roc_auc = roc_auc_score(df_val[target_col], y_pred) 

    return roc_auc

def split_dataset_by_target(data, target, n):
    """
    Разделить датасет на n групп, с одинаковым положительным таргетом и разными негативными.
    :param data: pandas.DataFrame, содержащий данные для разделения.
    :param target: str, описывающий положительный таргет.
    :param n: количество групп для разделения (и, соответственно, количество моделей).
    :return: список, содержащий n кортежей из данных для обучения.
    """
    sorted_data = data.sort_values(target, kind='mergesort')
    positive_set_index = sorted_data.index[sorted_data[target] == 1]
    positive_set_size = len(positive_set_index)
    negative_set_index = sorted_data.index[sorted_data[target] == 0]
    negative_set_size = len(negative_set_index)
    negative_sets = []
    for i in range(n-1):
        negative_sets.append(negative_set_index[i*(negative_set_size//n):(i+1)*(negative_set_size//n)])
    negative_sets.append(negative_set_index[(n-1)*(negative_set_size//n):])
    splitted_data = []
    for i in range(n):
        negative_index = negative_sets[i]
        train_data_index = positive_set_index.append(negative_index)
        # test_data_index = negative_set_index.difference(negative_index)
        train_data = data.loc[train_data_index]
        # test_data = data.loc[test_data_index]
        splitted_data.append(train_data)
    return splitted_data
    
class ModelBuilder(object):
    def __init__(self, cat_cols, features_name, ml_cfg, target_col='target', params=None):
        
        self.features = features_name
        self.cat_cols = cat_cols
        self.target_col = target_col
        self.cfg = ml_cfg
        selector = FeatureSelector(features=features_name)
        enc = CatBoostEncoder(cols=cat_cols)
        lgbm = LGBMClassifier()
        self.model = Pipeline([('sel', selector), ('enc', enc), ('lgbm', lgbm)])      
        self.model.set_params(**params) 
        
        
    def train(self, df_train):
        model = self.model.fit(df_train, df_train[self.target_col])
        return model
        
    def select_features(self, df_train, df_val):

        model = self.train(df_train)
        transformer = Pipeline(model.steps[:-1])
        model = model.steps[-1][-1]
        transformed_val_data = transformer.transform(df_val)

        print('start feature select...')
        perm = PermutationImportance(model, random_state=42) \
                                    .fit(transformed_val_data, df_val[self.target_col])
        print('end feature select...')

        feature_importances = list(zip(transformed_val_data.columns, perm.feature_importances_))
        feature_importances.sort(key=lambda x: x[1], reverse=True)
        selected_featurs = [name_col for name_col, value in feature_importances if value > 0]
        print(f'selected {len(selected_featurs)} out of {len(df_train.columns)}')
        
        cat_cols = list(set(self.cat_cols) & set(selected_featurs))
        selected_features = {'sel__features': selected_featurs,'enc__cols': cat_cols}
        return selected_features
    
    def params_tuning(self, df_train, df_val, model):
        print('start tuning...')
        study = optuna.create_study(direction='maximize')
        transformer = Pipeline(model.steps[:-1])
        study.optimize(lambda trial: objective(trial, transformer, df_train, df_val),
                       n_jobs=1, n_trials=self.cfg['n_trails'])
        print('end tuning...')

        best_params = study.best_params
        best_score = study.best_value
        best_params = {f'lgbm__{k}':v for k,v in best_params.items()}
        # self.model.set_params(**best_params)
        print(f'best params: {best_params}')
        print(f'best score: {best_score}')
        return best_params
    
    def build(self, df):
        print('start build model...')
        df_train, df_test = train_test_split(df, random_state=42, test_size=self.cfg['test_size'],
                                            shuffle=True, stratify=df[self.target_col])
        
        test = split_dataset_by_target(df_train, self.target_col, self.cfg['n_split_df'])
        
        models = []
        
        for df in test:
            df_train, df_val = train_test_split(df, random_state=42, test_size=self.cfg['test_size'],
                                            shuffle=True, stratify=df[self.target_col])
            
            features = self.select_features(df_train, df_val)
            model = deepcopy(self.model)
            model.set_params(**features)
            best_params = self.params_tuning(df_train, df_val, model)
            model.set_params(**best_params)

            model.fit(df_train, df_train[self.target_col])

            y_pred = model.predict(df_val)
            metrics = roc_auc_score(df_val[self.target_col], y_pred)
            print(f'roc_auc on validate data = {metrics}')
            models.append(model)
        preds = []
        for m in models:
            preds.append(m.predict_proba(df_test)[:, 1])
        result = pd.DataFrame(preds).mean()
        print('metrics on test data:')
        evaluate_metrics(df_test['target'], result)
        return models

# 3. Create and get trainable model

In [27]:
exclude_cols = ['target', 'client_id', 'report_date']
features = df.columns.tolist()
features = list(set(features) - set(exclude_cols))
category_cols = list(set(features) & set(category_cols))
default_params = {'lgbm__n_estimators': 100, 'lgbm__max_depth': 10, 
                  'lgbm__random_state': 42, 'lgbm__n_jobs': -1}
ml_cfg =  {'n_trails': 100, 'test_size': 0.3, 'n_split_df': 3}

In [28]:
df['target'] = df['target'].astype(int)

In [29]:
builder = ModelBuilder(cat_cols = category_cols, features_name=features, ml_cfg=ml_cfg, params=default_params)

In [30]:
%%time
models = builder.build(df)

start build model...
start feature select...


[I 2023-09-10 01:02:22,024] A new study created in memory with name: no-name-1841985a-57a0-4230-bb48-c93d956dbd6c


end feature select...
selected 85 out of 342
Finished loading model, total used 100 iterations
start tuning...


C:\Users\Artem\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-09-10 01:02:22,472] Trial 0 finished with value: 0.807090147586757 and parameters: {'num_leaves': 177, 'learning_rate': 0.004480001263134817, 'feature_fraction': 0.3102741342546008, 'bagging_fraction': 0.9701755396059921, 'bagging_freq': 4, 'max_depth': 3, 'min_child_samples': 53, 'min_sum_hessian_in_leaf': 9.07594152588438, 'is_unbalance': True, 'random_state': 42}. Best is trial 0 with value: 0.807090147586757.


ROC AUC: 0.807090147586757; Accuracy: 0.9142590866728798; Precision: 0.0; Recall: 0.0; F1 Score: 0.0; PR AUC: 0.29901968657359324; Log Loss: 0.2813556007874963


...

ROC AUC: 0.9079462941645267; Accuracy: 0.9143389199255121; Precision: 0.0; Recall: 0.0; F1 Score: 0.0; PR AUC: 0.6336849630280837; Log Loss: 0.2641986105264968


C:\Users\Artem\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-09-10 01:05:08,294] Trial 92 finished with value: 0.9180299743203755 and parameters: {'num_leaves': 187, 'learning_rate': 0.014392528833066335, 'feature_fraction': 0.13496648152122087, 'bagging_fraction': 0.9876052174735012, 'bagging_freq': 2, 'max_depth': 12, 'min_child_samples': 28, 'min_sum_hessian_in_leaf': 2.752827497441662, 'is_unbalance': True, 'random_state': 42}. Best is trial 92 with value: 0.9180299743203755.


ROC AUC: 0.9180299743203755; Accuracy: 0.9143389199255121; Precision: 0.0; Recall: 0.0; F1 Score: 0.0; PR AUC: 0.6323204747095775; Log Loss: 0.26071392970135004


[I 2023-09-10 01:05:08,791] Trial 93 pruned. Trial was pruned at iteration 0.
[I 2023-09-10 01:05:09,294] Trial 94 pruned. Trial was pruned at iteration 0.
[I 2023-09-10 01:05:09,796] Trial 95 pruned. Trial was pruned at iteration 0.
[I 2023-09-10 01:05:10,304] Trial 96 pruned. Trial was pruned at iteration 2.
[I 2023-09-10 01:05:10,814] Trial 97 pruned. Trial was pruned at iteration 1.
[I 2023-09-10 01:05:11,318] Trial 98 pruned. Trial was pruned at iteration 0.
[I 2023-09-10 01:05:11,824] Trial 99 pruned. Trial was pruned at iteration 0.


end tuning...
best params: {'lgbm__num_leaves': 187, 'lgbm__learning_rate': 0.014392528833066335, 'lgbm__feature_fraction': 0.13496648152122087, 'lgbm__bagging_fraction': 0.9876052174735012, 'lgbm__bagging_freq': 2, 'lgbm__max_depth': 12, 'lgbm__min_child_samples': 28, 'lgbm__min_sum_hessian_in_leaf': 2.752827497441662, 'lgbm__is_unbalance': True, 'lgbm__random_state': 42}
best score: 0.9180299743203755
roc_auc on validate data = 0.7214756929071107
metrics on test data:
ROC AUC: 0.8865325071780408; Accuracy: 0.9439704865114135; Precision: 0.2565217391304348; Recall: 0.45038167938931295; F1 Score: 0.3268698060941828; PR AUC: 0.23645772891042358; Log Loss: 0.19516081567686666
CPU times: total: 25min 12s
Wall time: 3min 3s


In [31]:
import pickle

In [38]:
with open(f'model.pickle', 'wb') as f:
    pickle.dump(models, f)

In [40]:
# with open(f'model.pickle', 'rb') as f:
#     md = pickle.load(f)

# 4. Submission file

In [229]:
def inference_model(df, models, prob_col='score'):
    preds = []
    for m in models:
        preds.append(m.predict_proba(df_test)[:, 1])
    df[prob_col] = pd.DataFrame(preds).mean()
    return df

In [ ]:
test = pd.read_csv(...)

In [ ]:
test = inference_model(test, models)

In [ ]:
sample_submission = test.loc[:, ['report_date', 'client_id', 'score']]

In [ ]:
sample_submission.to_csv('result_novichki.csv')